In [26]:
import json
from datetime import datetime
from itertools import groupby
from collections import defaultdict
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px
import urllib.request
import json

In [2]:
plt.rcParams['figure.figsize'] = [15, 8]

In [56]:
history_data_path = 'reddit_crypto_stat.csv'
recent_data_path = 'reddit_crypto_history.csv'

df_history = pd.read_csv(history_data_path)
df_recent = pd.read_csv(recent_data_path)
df = pd.concat([df_history, df_recent])
keywords = df.keyword.unique()
df = df.set_index(['keyword','date'])
df = df.sort_index()
df.head()

num_keyword_posts  num_comments  num_keyword_comments
keyword date                                                           
ADA     21-01-28              116.0         783.0                  23.0
        21-01-29              132.0         586.0                  11.0
        21-01-30              118.0         604.0                  24.0
        21-01-31              192.0        1010.0                  65.0
        21-02-01              130.0        2686.0                  40.0

In [57]:
keywords

array(['ADA', 'AVAX', 'Aave', 'Avalanche', 'BCH', 'BNB', 'BSV', 'BTC',
       'BUSD', 'Binance', 'Binance Coin', 'Binance USD', 'Bitcoin',
       'Bitcoin Cash', 'Bitcoin SV', 'CEL', 'CRO', 'Cardano', 'Chainlink',
       'Cosmos', 'Crypto.com', 'Crypto.com Coin', 'DASH', 'DCR', 'DFI',
       'DOGE', 'DOT', 'Dai', 'Dash', 'Decred', 'DefiChain', 'Dogecoin',
       'EGLD', 'EOS', 'ETH', 'Elrond', 'Ethereum', 'Ethereum Classic',
       'FIL', 'FTT', 'FTX Token', 'Filecoin', 'GRT', 'HT', 'IOTA', 'KSM',
       'Kusama', 'LEO', 'LTC', 'Litecoin', 'MIOTA', 'MKR', 'Monero',
       'NEM', 'Neo', 'Polkadot', 'REV', 'Ripple', 'SNX', 'SOL', 'Solana',
       'Stellar', 'SushiSwap', 'Synthetix', 'Synthetix Network Token',
       'TRX', 'Tether', 'Tezos', 'The Graph', 'Theta', 'Tron', 'UMA',
       'UNI', 'USD Coin', 'USDC', 'USDT', 'Uniswap', 'VeChain', 'XEM',
       'XLM', 'XMR', 'XRP', 'XTZ', 'YFI', 'Yearn.Finance', 'ZCash', 'ZEC',
       'ETC', 'Huobi Token', 'Revain'], dtype=object)

In [106]:
keyword = "TRX"

url = f"https://www.binance.com/api/v1/klines?symbol={keyword}BUSD&interval=1d"
data = urllib.request.urlopen(url)

data = json.load(data)

In [107]:
history_quotes = [[datetime.fromtimestamp(int(str(_[0])[:10])), _[4], _[5]] for _ in data]
df_hist = pd.DataFrame(data=history_quotes, columns=["ts","price","volume"])
df_hist = df_hist.set_index("ts")
df_hist = pd.DataFrame(data=MinMaxScaler().fit_transform(df_hist),index=df_hist.index,columns=df_hist.columns)
df_hist.index = df_hist.index.date
df_hist.head()

,price,volume
2019-12-02,0.049419,0.005580
2019-12-03,0.049347,0.005994
2019-12-04,0.042598,0.008454
2019-12-05,0.044340,0.017316
2019-12-06,0.045646,0.016147


In [108]:

df_k = df.loc[keyword]
df_k = pd.DataFrame(data=MinMaxScaler().fit_transform(df_k),index=df_k.index, columns=df_k.columns)
df_k.index = pd.to_datetime(df_k.index,yearfirst=True)
df_k.head()


,num_keyword_posts,num_comments,num_keyword_comments
date,,,
2021-01-28,0.059172,0.015697,NaN
2021-01-29,0.159763,0.088643,0.059701
2021-01-30,0.828402,0.221607,0.298507
2021-01-31,0.295858,0.125577,0.059701
2021-02-01,0.284024,0.401662,0.194030


In [111]:
latest_ts = df_k.index[0]
df_hist = df_hist.loc[latest_ts:]

fig = px.line(x=df_k.index, y=df_k.num_comments.rolling(5, min_periods=1).mean(),title=keyword)
fig.add_scatter(x=df_k.index, y=df_k.num_comments.rolling(5, min_periods=1).mean(), name='Comments')
fig.add_scatter(x=df_k.index, y=df_k.num_keyword_posts.rolling(5, min_periods=1).mean(), name='Keyword posts')
fig.add_scatter(x=df_k.index, y=df_k.num_keyword_comments.rolling(5, min_periods=1).mean(), name='Keyword comments')
fig.add_scatter(x=df_hist.index, y=df_hist.price, name='Price')
fig.add_scatter(x=df_hist.index, y=df_hist.volume.rolling(5, min_periods=1).mean(), name='Volume')
fig.show()